In [1]:
!pip install google-cloud-speech


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# ==================================================================================
# Google Cloud STT API를 활용한 음성 파일 화자 분리 및 전사 스크립트
# ==================================================================================
#
# [ 주요 기능 ]
# 1. Google Cloud Storage (GCS) 연동: 로컬 파일이 아닌, GCS에 업로드된 오디오 파일을 처리합니다.
#    - 1분 이상의 긴 오디오 파일 처리에 적합한 'long_running_recognize' 메서드를 사용합니다.
# 2. 화자 분리 (Speaker Diarization): 음성에 포함된 2명의 화자를 자동으로 식별하고,
#    각 발화 내용 앞에 [화자 1], [화자 2]와 같이 태그를 붙여 출력합니다.
#    - 보이스피싱 통화처럼 2명의 대화로 구성된 음성 분석에 최적화되어 있습니다.
# 3. 자동 구두점 및 한국어 최적화: 결과 텍스트에 쉼표, 마침표 등을 자동으로 추가하며,
#    한국어('ko-KR') 음성 인식을 수행합니다.
#
# [ 사용 방법 ]
# 1. `.env` 파일 설정: 스크립트와 같은 위치에 .env 파일을 만들고, 그 안에
#    `GOOGLE_APPLICATION_CREDENTIALS="여기에/서비스/계정/json/키파일_경로.json"` 형식으로
#    인증 키 파일 경로를 지정합니다.
# 2. GCS 오디오 업로드: 분석할 오디오 파일(예: audio.mp3)을 본인의 GCS 버킷에 업로드합니다.
# 3. 스크립트 내 변수 수정: 코드의 `GCS_BUCKET_NAME`과 `GCS_AUDIO_FILENAME` 변수를
#    본인의 GCS 버킷 이름과 업로드한 파일 이름으로 수정합니다.
# 4. 스크립트 실행: 터미널에서 `python your_script_name.py`를 실행하면, GCS의 파일을
#    분석하여 화자가 분리된 텍스트 결과를 출력합니다.
#
# ==================================================================================


import os
from dotenv import load_dotenv
from google.cloud import speech
from google.api_core.exceptions import GoogleAPICallError

# --- 1. 환경 설정 ---
print("환경 설정을 시작합니다...")

# .env 파일에서 환경 변수 로드
load_dotenv()
google_creds_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if not google_creds_path or not os.path.exists(google_creds_path):
    raise FileNotFoundError(
        f"❌ .env 파일에 GOOGLE_APPLICATION_CREDENTIALS 경로가 없거나 파일이 존재하지 않습니다: '{google_creds_path}'"
    )

# Google Cloud 인증 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_creds_path
print("✅ Google Cloud 인증 정보 로드 완료")

# GCS 버킷 및 파일 정보 (이 부분은 사용자가 직접 업로드 후 설정해야 함)
GCS_BUCKET_NAME = "woogawooga_project_stt_audio"  # 본인의 GCS 버킷 이름
GCS_AUDIO_FILENAME = "audio.mp3"  # GCS 버킷에 업로드한 파일 이름
GCS_URI = f"gs://{GCS_BUCKET_NAME}/{GCS_AUDIO_FILENAME}"

print(f"✅ 처리할 오디오 파일(GCS): {GCS_URI}")

# --- 2. Google Cloud STT API 호출 ---
try:
    client = speech.SpeechClient()

    # 오디오 설정
    audio = speech.RecognitionAudio(uri=GCS_URI)

    # 화자 분리 포함 인식 설정
    diarization_config = speech.SpeakerDiarizationConfig(
        enable_speaker_diarization=True,
        min_speaker_count=2,
        max_speaker_count=2,  # 보이스피싱은 보통 2명이므로 고정
    )

    config = speech.RecognitionConfig(
        language_code="ko-KR",
        # sample_rate_hertz=48000, # GCS 사용 시 자동 감지되므로 주석 처리 가능
        enable_automatic_punctuation=True,
        diarization_config=diarization_config,
    )

    print("\n🔹 Google Cloud에 음성 인식(장기 실행) 요청 중...")
    operation = client.long_running_recognize(config=config, audio=audio)

    print("🔹 작업 완료 대기 중... (파일 길이에 따라 수 분 소요될 수 있음)")
    response = operation.result(timeout=600)  # 10분 타임아웃

    # --- 3. 결과 처리 및 출력 ---
    if not response or not response.results:
        print("\n❌ 음성을 인식할 수 없거나 유효한 결과가 없습니다.")
    else:
        # 마지막 결과에만 화자 분리 정보가 포함됨
        final_result = response.results[-1]
        alternative = final_result.alternatives[0]

        print("\n" + "=" * 50)
        print("          Google STT 화자 분리 결과            ")
        print("=" * 50)

        current_speaker_tag = None
        transcript_segment = ""

        for word_info in alternative.words:
            speaker_tag = word_info.speaker_tag

            if current_speaker_tag is None:
                current_speaker_tag = speaker_tag

            # 화자가 바뀌면 이전 대화 출력
            if speaker_tag != current_speaker_tag:
                print(f"[화자 {current_speaker_tag}]: {transcript_segment.strip()}")
                # 초기화
                transcript_segment = ""
                current_speaker_tag = speaker_tag

            transcript_segment += word_info.word + " "

        # 마지막 발화 내용 출력
        if transcript_segment:
            print(f"[화자 {current_speaker_tag}]: {transcript_segment.strip()}")

        print("=" * 50)
        print("\n✅ 작업이 성공적으로 완료되었습니다.")

except GoogleAPICallError as e:
    print(f"\n❌ Google Cloud API 호출 중 오류가 발생했습니다: {e}")
except Exception as e:
    print(f"\n❌ 알 수 없는 오류가 발생했습니다: {e}")

✅ 설정 완료: 경로와 언어가 지정되었습니다.


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

client = speech.SpeechClient()

print("✅ 인증 성공: API를 사용할 준비가 되었습니다.")

✅ 인증 성공: API를 사용할 준비가 되었습니다.


In [ ]:
# ==================================================================================
# Google Cloud STT - 상세 화자 분리 결과 처리 스크립트
# ==================================================================================
#
# [ 주요 기능 ]
# 1. 상세 결과 처리: API가 반환하는 모든 'results' 배열을 순회하며, 각 문단(segment)별로
#    화자를 분리하여 전체 대화 내용을 재구성합니다.
#    - 이전 방식이 마지막 'result' 하나만 사용했던 것과 달리, 이 코드는 더 세분화된
#      전체 변환 결과를 보여줍니다.
# 2. 명시적 설정 적용: 오디오 샘플링 레이트(sample_rate_hertz)를 48000으로 명시적으로
#    설정하여 특정 오디오 환경에서의 인식 정확도를 높이려 시도합니다.
# 3. GCS 기반 장기 실행: Google Cloud Storage(GCS)에 저장된 긴 오디오 파일에 대한
#    비동기식 음성 인식을 수행합니다.
#
# [ 사용 방법 ]
# 1. 이 스크립트를 실행하기 전에 Google Cloud 인증 및 GCS 파일 업로드가
#    선행되어야 합니다.
# 2. 코드 내의 bucket_name과 audio_file_name_in_gcs 변수를 실제 GCS 환경에 맞게
#    수정한 후 실행합니다.
# 3. 실행이 완료되면, 터미널에 화자별로 정리된 전체 대화 내용이 출력됩니다.
#
# ==================================================================================



bucket_name = "woogawooga_project_stt_audio"
audio_file_name_in_gcs = "audio.mp3"
gcs_uri = f"gs://{bucket_name}/{audio_file_name_in_gcs}"
audio = speech.RecognitionAudio(uri=gcs_uri)


print("✅ 상세 설정(샘플링 레이트 등)으로 새로운 Config를 생성합니다...")

diarization_config = speech.SpeakerDiarizationConfig(
    enable_speaker_diarization=True,
    min_speaker_count=2,
    max_speaker_count=2,
)

config = speech.RecognitionConfig(
    language_code="ko-KR",
    sample_rate_hertz=48000, 
    enable_automatic_punctuation=True,  
    diarization_config=diarization_config,
)


print(f"GCS 파일('{gcs_uri}')에 대한 장기 실행 인식을 다시 요청합니다...")
operation = client.long_running_recognize(config=config, audio=audio)

print(
    "작업이 완료될 때까지 기다립니다. 파일 길이에 따라 몇 분 정도 소요될 수 있습니다..."
)
response = operation.result(timeout=600)  #


if response and response.results:
    print("\n--- 전체 화자 분리 변환 결과 ---")

    for result in response.results:
        alternative = result.alternatives[0]
        if not alternative.words:
            continue

        current_speaker = alternative.words[0].speaker_tag
        transcript_segment = ""

        for word_info in alternative.words:
            if word_info.speaker_tag == current_speaker:
                transcript_segment += " " + word_info.word
            else:
                if transcript_segment.strip():
                    print(f"화자 {current_speaker}: {transcript_segment.strip()}")

                current_speaker = word_info.speaker_tag
                transcript_segment = " " + word_info.word

        if transcript_segment.strip():
            print(f"화자 {current_speaker}: {transcript_segment.strip()}")

    print("---------------------------------")

else:
    print("음성을 인식할 수 없거나, 유효한 결과가 없습니다.")

✅ 상세 설정(샘플링 레이트 등)으로 새로운 Config를 생성합니다...
GCS 파일('gs://woogawooga_project_stt_audio/audio.mp3')에 대한 장기 실행 인식을 다시 요청합니다...
작업이 완료될 때까지 기다립니다. 파일 길이에 따라 몇 분 정도 소요될 수 있습니다...

--- 전체 화자 분리 변환 결과 ---
화자 0: 여보세요 주시겠습니까 네 안녕하세요 지금 첨단범죄수사팀 국민 1인당 장점이자 없어요 네 언니가 연락 드렸는데요
화자 0: 41 42 세대는 남성 김철수 왕십리 아니 다름이 아니고요 얼마든지 검찰에서 금융 사기 집단을 검사했습니다 입금가 현장에서 1인용으로 대명통상 하고 국민의 통장이 발견돼서 지금 청소하고 있습니다 일할 때는 이유가 직접 계산하셔서 김천 씨한테 선물 받고 있는지 아니면 볼 일이 좀 있어서요 오늘 당신 회사 분인지 지금 이 부분을 좀 하라고 했습니다 그런 거 영상 적 없는데요
화자 0: 별일 없으시고요 그러면 일단 받겠습니다 누구한테 통장을 만들어 드릴 거나 영미를 영동선 수도 있으니까 사이에 주민등록증이나 면허증 분실 한 적이 있습니까 2014년 3월 3일에 경기도 광명시 철산 농협으로 꼼이네 통화는 괜찮아 사실 있습니다 그럼 본인이 하고 통장을 직접 결제를 하셨더니 아니면 얼마나 맛있던지 저희가 범죄현장에서 때문에 시켰습니다 직접적으로 함께 하셔서 사용하는 통장에 돈 좀 더 처리할 겁니다 네 몸이 직접 하셔서 주세요
화자 0: 어디 가서 해야 되나요? 내가 직접 처리해 주세요 금융권 사고요 창고만 사용하신다 사용하신다 이렇게요 본인이 직접 거래하는 그래 하는 거야. 사용하시고요 사용하십니까 아니요 지금 맛있게 마시고 사용하는데 아지트를 했으니까 지키는 몇 개 있습니까? 사용하십니까? 인터넷뱅킹이 스마트폰뱅킹 하는데요 스마트폰뱅킹 안 하는데.
화자 0: 아 그럼 제일 마지막 마지막 글자는 언제 가십니까 오늘 아침이야 아내 마지막 그래가지고 잔액이 얼마 정도 있어요. 정상입니까 70만 원 적